<a href="https://colab.research.google.com/github/jacksonkarel/selfmodifai/blob/main/notebooks/gpt4_agent_modify_alpaca_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai transformers

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
import json
import openai

keys = json.load(open('keys.json'))
openai.api_key = keys['openai']

In [ ]:
import os
os.environ["GIT_EMAIL"] = keys["git_email"]
os.environ["GIT_USERNAME"] = keys["git_username"]

In [ ]:
import os

!rm -rf /root/.ssh

!mkdir /root/.ssh

!tar xvzf ssh.tar.gz

!cp ssh-colab/* /root/.ssh && rm -rf ssh-colab && rm -rf ssh.tar.gz
!chmod 700 .shh

!touch /root/.ssh/known_hosts
!ssh-keyscan github.com >> /root/.ssh/known_hosts
!chmod 644 /root/.ssh/known_hosts

!git config --global user.email GIT_EMAIL
!git config --global user.name GIT_USERNAME
!git clone git@github.com:jacksonkarel/selfmodifai-alpaca-lora.git
os.chdir("selfmodifai-alpaca-lora")

In [ ]:
access_token = keys["github_access_token"]

!git config --global user.email GIT_EMAIL
!git config --global user.name GIT_USERNAME

os.system(f"git clone https://{access_token}@github.com/jacksonkarel/selfmodifai-alpaca-lora.git")
os.chdir("selfmodifai-alpaca-lora")

In [ ]:
with open('messages.json') as json_file:
    messages = json.load(json_file)

messages.pop()

with open('messages.json', 'w') as outfile:
    json.dump(messages, outfile)

In [ ]:
!git restore messages.json

In [ ]:
!rm messages.json

In [ ]:
import re
import json
from transformers import pipeline
from openai.error import InvalidRequestError

def update_messages(content, role, messages):
    new_message = {"role": role, "content": content}
    messages.append(new_message)
    with open('messages.json', 'w') as outfile:
        json.dump(messages, outfile)
    
    print(f"Step: {content}")
    return messages

def handle_too_long_context(messages):
    print("too long context")
    messages = messages[:-1]
    full_context = "Condense the information from the following past conversation between us. Keep all of the information that is relevant to the task at hand and remove all that is not:\n"
    for message in messages[1:]:
        role = message["role"]
        content = message["content"]

        full_context += (f"{role}: {content}\n\n")
    
    print(full_context)
    system_turn = {'role': 'system', 'content': 'You are part of an agent that is modifying the code of the model Alpaca-LoRA. The agent is in the Alpaca-LoRA directory. When you write code, that code will be executed and the output will be sent back to you.'}
    
    less_messages = [system_turn, {'role': 'user', 'content': full_context}]
    
    try:
        response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=less_messages
                )
        less_messages = [system_turn, {'role': 'assistant', 'content': response["choices"][0]["message"]["content"]}]

    except InvalidRequestError as e:
            # Check if the error message matches the context length issue
            if "maximum context length" in str(e):
                 response, less_messages = handle_too_long_context(messages)

            else:
                # Re-raise the exception if it's not what we're looking for
                raise e
    
    return response, less_messages
    
def gpt4_agent():
    with open('messages.json') as json_file:
        messages = json.load(json_file)

    while True:
        
        try:
            response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages
            )
        except InvalidRequestError as e:
            # Check if the error message matches the context length issue
            if "maximum context length" in str(e):
                response, messages = handle_too_long_context(messages)
                
                with open('messages.json', 'w') as outfile:
                    json.dump(messages, outfile)

            else:
                # Re-raise the exception if it's not what we're looking for
                raise e

        response_content = response["choices"][0]["message"]["content"]
        
        messages = update_messages(response_content, "assistant", messages)
        
        # Define the regular expression pattern
        pattern = r'```bash\n(.*?)\n```'

        matches = re.findall(pattern, response_content, re.DOTALL)

        python_content = re.search(r'```python\n(.*?)\n```', response_content)

        python_prompt = "Write bash commands to implement those changes in the Python files."

        bash_response = "Create bash commands that implement those changes. Give me them one by one."
        if matches:
            content = ""
        # matches is now a list of all bash commands in the string
            for bash_command in matches:
                content += f"{bash_command}:\n"
                stream = os.popen(bash_command)
                
                content += stream.read()
            
            
            if python_content:
                content = python_prompt

            elif not content:
                content = "Ok, did that"
        
        elif python_content:
            content = python_prompt

        elif "?" not in response_content:
            content = bash_response

        else:
            classifier = pipeline("zero-shot-classification")
            labels = ["suggestion for what to do next", "question"]
            results = classifier(sequences=response_content, candidate_labels=labels, hypothesis_template="This text is a {}") 

            if results["labels"][0] == "suggestion for what to do next":
                content = bash_response
            
            else:
                content = "My goal is to improve the model architecture of Alpaca-LoRA to make it a more powerful language model. Find the answer to that question in that context. If you can't, try another step in improving the language model."
            
        
        messages = update_messages(content, "user", messages)

In [ ]:
gpt4_agent()